This is a service notebook to join together in one dataset all sentiment analysis data from different models and libraries plus quick review and analysis of usability.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
Data='/home/kate/Projects/eva/Data'
Messages_filename='Messages.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

sa_dostoevsky_filename='SentimentAnalysis/Dostoevsky/sa_dostoevsky.csv'
sa_dostoevsky_full_filename=os.path.join(Data, sa_dostoevsky_filename)

sa_tatyana_filename='SentimentAnalysis/tatyana/sa_tatyana.csv'
sa_tatyana_full_filename=os.path.join(Data, sa_tatyana_filename)

sa_twitter_filename='SentimentAnalysis/twitter/sa_twitter.csv'
sa_twitter_full_filename=os.path.join(Data, sa_twitter_filename)

sa_inappropriate_filename='SentimentAnalysis/inappropriate/sa_inappropriate.csv'
sa_inappropriate_full_filename=os.path.join(Data, sa_inappropriate_filename)

In [3]:
Messages = pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False) 

In [5]:
sa_dostoevsky = pd.read_csv(sa_dostoevsky_full_filename, error_bad_lines=False, index_col=False)
len(sa_dostoevsky)

9455613

In [6]:
sa_dostoevsky['dostoevsky'].unique()

array(['neutral', 'negative', 'positive', 'skip', 'speech'], dtype=object)

In [6]:
sa_dostoevsky['dostoevsky']=sa_dostoevsky['dostoevsky'].str.upper()

In [8]:
sa_dostoevsky['dostoevsky'].unique()

array(['NEUTRAL', 'NEGATIVE', 'POSITIVE'], dtype=object)

In [7]:
sa_dostoevsky['dostoevsky_n']=['NEUTRAL' if ((x=='SKIP') or (x=='SPEECH')) else x for x in sa_dostoevsky['dostoevsky']]

In [10]:
sa_dostoevsky['dostoevsky_n'].unique()

array(['NEUTRAL', 'NEGATIVE', 'POSITIVE'], dtype=object)

In [4]:
sa_tatyana = pd.read_csv(sa_tatyana_full_filename, error_bad_lines=False, index_col=False)
len(sa_tatyana)

9455613

In [11]:
sa_tatyana.head()

,Message_Id,tatyana
0,90651956,POSITIVE
1,90651971,NEGATIVE
2,90632653,NEUTRAL
3,90639753,POSITIVE
4,90640548,POSITIVE


In [12]:
sa_twitter = pd.read_csv(sa_twitter_full_filename, error_bad_lines=False, index_col=False)
len(sa_twitter)

9455613

In [13]:
sa_twitter.head()

,Message_Id,twitter
0,90651956,POSITIVE
1,90651971,NEGATIVE
2,90632653,NEUTRAL
3,90639753,POSITIVE
4,90640548,POSITIVE


In [9]:
Messages=pd.merge(Messages, sa_dostoevsky, left_on='Message_Id', right_on='Message_Id', how='left')

In [10]:
len(Messages[Messages['dostoevsky'].isnull()])

0

In [5]:
Messages=pd.merge(Messages, sa_tatyana, left_on='Message_Id', right_on='Message_Id', how='left')

In [6]:
len(Messages[Messages['tatyana'].isnull()])

0

In [18]:
Messages=pd.merge(Messages, sa_twitter, left_on='Message_Id', right_on='Message_Id', how='left')

In [19]:
len(Messages[Messages['twitter'].isnull()])

0

In [20]:
Messages.tail()

,Message_Id,Author_Id,message,dostoevsky,dostoevsky_n,tatyana,twitter
9455608,103135891,0,"Я школу закончила 30 лет назад, до сих пор не ...",NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE
9455609,103135894,0,"Знаю точно, что современные пищевые производст...",NEGATIVE,NEGATIVE,NEUTRAL,NEUTRAL
9455610,103135900,746948,"тут соглашусь, однако СССР тут не стоит выделя...",NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE
9455611,103135922,746948,"Кстати, в последнее время по нему очень заметн...",NEGATIVE,NEGATIVE,POSITIVE,POSITIVE
9455612,103135938,0,"Так же и с парниковыми овощами, а уж тем более...",NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL


In [22]:
Messages[Messages['twitter']!=Messages['tatyana']].head()

,Message_Id,Author_Id,message,dostoevsky,dostoevsky_n,tatyana,twitter


Tatyana/rubert-base-cased-sentiment-new (trained on customer reviews) and blanchefort/rubert-base-cased-sentiment (trained on twits) produced the same result!

In [23]:
Messages['isAnonymous']=['Yes' if x == 0 else 'No' for x in Messages['Author_Id']]

In [25]:
Messages[Messages['Author_Id']==0].head()

,Message_Id,Author_Id,message,dostoevsky,dostoevsky_n,tatyana,twitter,isAnonymous
229,42629245,0,Из бумажных лент кто-нибудь мастерит? Типа так...,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,Yes
258,43445105,0,как здорово-то!!! обязательно сплагиатим у вас...,POSITIVE,POSITIVE,POSITIVE,POSITIVE,Yes
260,43499521,0,"Девочки! просьба! Накидайте, пожалуйста, мне с...",NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,Yes
263,43524131,0,"Ой!!! Я совсем забыла указать, что как раз &qu...",NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,Yes
275,43827273,0,"Помогите, нужна поделка из метала на 23 февраля.",NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,Yes


In [28]:
Messages.groupby(['tatyana']).size().reset_index(name='counts')

,tatyana,counts
0,NEGATIVE,4131622
1,NEUTRAL,3230956
2,POSITIVE,2093035


In [29]:
Messages.groupby(['dostoevsky_n']).size().reset_index(name='counts')

,dostoevsky_n,counts
0,NEGATIVE,1966145
1,NEUTRAL,6881928
2,POSITIVE,607540


In [30]:
Messages.groupby(['dostoevsky']).size().reset_index(name='counts')

,dostoevsky,counts
0,NEGATIVE,1966145
1,NEUTRAL,6251703
2,POSITIVE,607540
3,SKIP,531484
4,SPEECH,98741


Dostoevsky is more conservative. There are less Negative and Positive and ~2 times more Neutral posts

In [26]:
Messages.groupby(['isAnonymous','tatyana']).size().reset_index(name='counts')

,isAnonymous,tatyana,counts
0,No,NEGATIVE,1772546
1,No,NEUTRAL,1473518
2,No,POSITIVE,1033676
3,Yes,NEGATIVE,2359076
4,Yes,NEUTRAL,1757438
5,Yes,POSITIVE,1059359


In [27]:
Messages.groupby(['isAnonymous','dostoevsky_n']).size().reset_index(name='counts')

,isAnonymous,dostoevsky_n,counts
0,No,NEGATIVE,830463
1,No,NEUTRAL,3136057
2,No,POSITIVE,313220
3,Yes,NEGATIVE,1135682
4,Yes,NEUTRAL,3745871
5,Yes,POSITIVE,294320


As expected, each model produced more Negative posts from anonymous authors. That's a known fact, the users are more open in their emotions when they are anonymous.

In [44]:
Messages[Messages['Message_Id']==55017545]

,Message_Id,Author_Id,message,dostoevsky,dostoevsky_n,tatyana,twitter,isAnonymous
5896336,55017545,116852,"дЕвы, снова включился отработанный сценарий. ""...",NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,No


In [46]:
Messages[Messages['Message_Id']==98973562]

,Message_Id,Author_Id,message,dostoevsky,dostoevsky_n,tatyana,twitter,isAnonymous
4953353,98973562,235,"Подлости нет? Это вы не работаете , наверное. ...",NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,No


More or less correctly. I wish I had sarcasm detection. 

## Inapropriate messages

In [5]:
sa_inappropriate = pd.read_csv(sa_inappropriate_full_filename, error_bad_lines=False, index_col=False)
len(sa_inappropriate)

9455613

In [6]:
Messages=pd.merge(Messages, sa_inappropriate, left_on='Message_Id', right_on='Message_Id', how='left')

In [7]:
len(Messages[Messages['inappropriate'].isnull()])

0

In [8]:
Messages.to_csv(Messages_full_filename, header=True, index=False)

In [11]:
Messages.columns

Index(['Message_Id', 'Parent_Id', 'date_time', 'Author_Id', 'author',
       'original_message', 'message', 'message_characters', 'message_words',
       'emojis', 'images', 'links', 'original_paragraphs', 'actual_paragraphs',
       ' avg_paragraph_characters', 'avg_paragraph_words', 'Topic_Id', 'Topic',
       'Chapter_Id', 'Chapter', 'cnt_childs', 'dostoevsky', 'tatyana',
       'inappropriate'],
      dtype='object')

In [14]:
Messages[((Messages['tatyana']=='NEGATIVE') & (Messages['inappropriate']=='Y'))]['Message_Id'].tail()

9455548    103125556
9455573    103127230
9455593    103133140
9455598    103133483
9455607    103135610
Name: Message_Id, dtype: int64

Mostly about using "bad" words (which is rare, because the forum is moderated) or just discussion of kids punishment, even in a negative way ("It's bad to beat a kid" is considered inapropriate)

In [15]:
Messages.groupby(['tatyana','inappropriate']).size().reset_index(name='counts').sort_values('counts', ascending=False)

,tatyana,inappropriate,counts
0,NEGATIVE,N,3187463
2,NEUTRAL,N,2899981
4,POSITIVE,N,1829028
1,NEGATIVE,Y,944159
3,NEUTRAL,Y,330975
5,POSITIVE,Y,264007


No practical use of this feature